In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score
import csv

# Load the datasets
train_data = pd.read_csv('/content/drive/MyDrive/train.csv')
asciified_data = pd.read_csv('/content/drive/MyDrive/train_ascii.csv')
test_data = pd.read_csv('/content/drive/MyDrive/test.csv')

# Merge the datasets based on ID
merged_data = pd.merge(asciified_data, train_data, on='ID')

Removing the first character, since the first character is ' ' in all of the sentences in test set

In [3]:
# Function to remove the first character of a string,
def remove_first_char(sentence):
    return sentence[1:]

# Apply the function to the 'Sentence' column
test_data['Sentence'] = test_data['Sentence'].apply(remove_first_char)

### Creating the vocabulary in char-level using both train and test set

In [4]:
# Function to tokenize each letter in a sentence
def letter_tokenization(sentence):
    return list(sentence)

# Apply letter tokenization to each sentence in both columns
merged_data['Tokenized_x'] = merged_data['Sentence_x'].apply(letter_tokenization)
merged_data['Tokenized_y'] = merged_data['Sentence_y'].apply(letter_tokenization)
test_data['Tokenized'] = test_data['Sentence'].apply(letter_tokenization)

In [5]:
import numpy as np

# Create vocabulary
char_to_index = {}
index_to_char = {}

# Add special tokens
char_to_index['<PAD>'] = 0
index_to_char[0] = '<PAD>'
char_to_index['<UNK>'] = 1
index_to_char[1] = '<UNK>'
char_to_index['<EOS>'] = 2
index_to_char[2] = '<EOS>'

# Function to add words to vocabulary
def add_to_vocab(chars):
    for char in chars:
        if char not in char_to_index:
            char_to_index[char] = len(char_to_index)
            index_to_char[len(char_to_index) - 1] = char

# Create vocabulary from tokenized input and labels
merged_data['Tokenized_x'].apply(add_to_vocab)
merged_data['Tokenized_y'].apply(add_to_vocab)
test_data['Tokenized'].apply(add_to_vocab)

# Add <EOS> token to the end of each sentence
merged_data['Tokenized_x'] = merged_data['Tokenized_x'].apply(lambda x: x + ['<EOS>'])
merged_data['Tokenized_y'] = merged_data['Tokenized_y'].apply(lambda x: x + ['<EOS>'])
test_data['Tokenized'] = test_data['Tokenized'].apply(lambda x: x + ['<EOS>'])

In [6]:
merged_data.head()

,ID,Sentence_x,Sentence_y,Tokenized_x,Tokenized_y
0,0,"sinif , havuz ve acik deniz calismalariyla , t...","sınıf , havuz ve açık deniz çalışmalarıyla , t...","[s, i, n, i, f, , ,, , h, a, v, u, z, , v, ...","[s, ı, n, ı, f, , ,, , h, a, v, u, z, , v, ..."
1,1,"bu standart , sualtinda kendini rahat hisseden...","bu standart , sualtında kendini rahat hisseden...","[b, u, , s, t, a, n, d, a, r, t, , ,, , s, ...","[b, u, , s, t, a, n, d, a, r, t, , ,, , s, ..."
2,2,"yapilan arastirmalar , ogrencilerin mevcut dal...","yapılan araştırmalar , öğrencilerin mevcut dal...","[y, a, p, i, l, a, n, , a, r, a, s, t, i, r, ...","[y, a, p, ı, l, a, n, , a, r, a, ş, t, ı, r, ..."
3,3,"pdic ogrencilerinde , psikolojik egitim ve yet...","pdıc öğrencilerinde , psikolojik eğitim ve yet...","[p, d, i, c, , o, g, r, e, n, c, i, l, e, r, ...","[p, d, ı, c, , ö, ğ, r, e, n, c, i, l, e, r, ..."
4,4,"pdic egitiminin sagladigi guven ve rahatlik , ...","pdıc eğitiminin sağladığı güven ve rahatlık , ...","[p, d, i, c, , e, g, i, t, i, m, i, n, i, n, ...","[p, d, ı, c, , e, ğ, i, t, i, m, i, n, i, n, ..."


In [7]:
test_data.head()

,ID,Sentence,Tokenized
0,0,tr ekonomi ve politika haberleri turkiye nin e...,"[t, r, , e, k, o, n, o, m, i, , v, e, , p, ..."
1,1,uye girisi,"[u, y, e, , g, i, r, i, s, i, , <EOS>]"
2,2,son guncelleme 12:12,"[s, o, n, , g, u, n, c, e, l, l, e, m, e, , ..."
3,3,Imrali Mit gorusmesi ihtiyac duyuldukca oluyor,"[I, m, r, a, l, i, , M, i, t, , g, o, r, u, ..."
4,4,Suriye deki silahli selefi muhalifler yeni kur...,"[S, u, r, i, y, e, , d, e, k, i, , s, i, l, ..."


In [8]:
# Convert tokens to indices
def tokens_to_indices(tokens):
    return [char_to_index[char] for char in tokens]

# Apply tokenization and indexing to the DataFrame
merged_data['Indexed_x'] = merged_data['Tokenized_x'].apply(tokens_to_indices)
merged_data['Indexed_y'] = merged_data['Tokenized_y'].apply(tokens_to_indices)

In [9]:
merged_data.head()

,ID,Sentence_x,Sentence_y,Tokenized_x,Tokenized_y,Indexed_x,Indexed_y
0,0,"sinif , havuz ve acik deniz calismalariyla , t...","sınıf , havuz ve açık deniz çalışmalarıyla , t...","[s, i, n, i, f, , ,, , h, a, v, u, z, , v, ...","[s, ı, n, ı, f, , ,, , h, a, v, u, z, , v, ...","[3, 4, 5, 4, 6, 7, 8, 7, 9, 10, 11, 12, 13, 7,...","[3, 129, 5, 129, 6, 7, 8, 7, 9, 10, 11, 12, 13..."
1,1,"bu standart , sualtinda kendini rahat hisseden...","bu standart , sualtında kendini rahat hisseden...","[b, u, , s, t, a, n, d, a, r, t, , ,, , s, ...","[b, u, , s, t, a, n, d, a, r, t, , ,, , s, ...","[24, 12, 7, 3, 22, 10, 5, 17, 10, 20, 22, 7, 8...","[24, 12, 7, 3, 22, 10, 5, 17, 10, 20, 22, 7, 8..."
2,2,"yapilan arastirmalar , ogrencilerin mevcut dal...","yapılan araştırmalar , öğrencilerin mevcut dal...","[y, a, p, i, l, a, n, , a, r, a, s, t, i, r, ...","[y, a, p, ı, l, a, n, , a, r, a, ş, t, ı, r, ...","[21, 10, 27, 4, 18, 10, 5, 7, 10, 20, 10, 3, 2...","[21, 10, 27, 129, 18, 10, 5, 7, 10, 20, 10, 13..."
3,3,"pdic ogrencilerinde , psikolojik egitim ve yet...","pdıc öğrencilerinde , psikolojik eğitim ve yet...","[p, d, i, c, , o, g, r, e, n, c, i, l, e, r, ...","[p, d, ı, c, , ö, ğ, r, e, n, c, i, l, e, r, ...","[27, 17, 4, 15, 7, 25, 23, 20, 14, 5, 15, 4, 1...","[27, 17, 129, 15, 7, 134, 133, 20, 14, 5, 15, ..."
4,4,"pdic egitiminin sagladigi guven ve rahatlik , ...","pdıc eğitiminin sağladığı güven ve rahatlık , ...","[p, d, i, c, , e, g, i, t, i, m, i, n, i, n, ...","[p, d, ı, c, , e, ğ, i, t, i, m, i, n, i, n, ...","[27, 17, 4, 15, 7, 14, 23, 4, 22, 4, 19, 4, 5,...","[27, 17, 129, 15, 7, 14, 133, 4, 22, 4, 19, 4,..."


## Training

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Model parameters
vocab_size = len(char_to_index)
embedding_dim = 100
hidden_units = 64

# Define model architecture
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, mask_zero=True),
    LSTM(units=hidden_units, return_sequences=True),
    Dense(units=vocab_size, activation='softmax')
])

In [11]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Pad sequences to a fixed length
max_length = 1200  # train ~ 1800, test ~ 1100
padded_input = pad_sequences(merged_data['Indexed_x'], maxlen=max_length, padding='post')
padded_label = pad_sequences(merged_data['Indexed_y'], maxlen=max_length, padding='post')

# Convert to numpy arrays
padded_input = np.array(padded_input)
padded_label = np.array(padded_label)

In [12]:
# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(padded_input, padded_label, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
1310/1310 [==============================] - 47s 30ms/step - loss: 0.7015 - accuracy: 0.8158 - val_loss: 0.1762 - val_accuracy: 0.9399
Epoch 2/5
1310/1310 [==============================] - 35s 27ms/step - loss: 0.1048 - accuracy: 0.9553 - val_loss: 0.1438 - val_accuracy: 0.9497
Epoch 3/5
1310/1310 [==============================] - 32s 25ms/step - loss: 0.0875 - accuracy: 0.9632 - val_loss: 0.1298 - val_accuracy: 0.9561
Epoch 4/5
1310/1310 [==============================] - 31s 24ms/step - loss: 0.0792 - accuracy: 0.9672 - val_loss: 0.1208 - val_accuracy: 0.9584
Epoch 5/5
1310/1310 [==============================] - 31s 24ms/step - loss: 0.0742 - accuracy: 0.9691 - val_loss: 0.1147 - val_accuracy: 0.9595


# Make Predictions

In [16]:
# Make predictions on a subset of the training set (first 2 examples)
train_predictions = model.predict(padded_input[0:4])
# Display examples
for idx in range(4):
    print("Example", idx+1)
    print("Input:", ''.join([index_to_char[i] for i in padded_input[idx] if i != 0]))  # Remove padding
    print("Label:", ''.join([index_to_char[i] for i in padded_label[idx] if i != 0]))  # Remove padding
    # Get predicted indices for the current example
    predicted_indices = train_predictions[idx].argmax(axis=1)
    # Remove padding and stop at <EOS> token
    predicted_sentence = []
    for i in predicted_indices:
        if i == 0:  # Stop at padding
            break
        if index_to_char[i] == '<EOS>':  # Stop at <EOS>
            break
        predicted_sentence.append(index_to_char[i])
    print("Prediction:", ''.join(predicted_sentence))
    print()

1/1 [==============================] - 0s 24ms/step
Example 1
Input: sinif , havuz ve acik deniz calismalariyla , tum dunyada gecerli , basarili bir standart olusturmustur . <EOS>
Label: sınıf , havuz ve açık deniz çalışmalarıyla , tüm dünyada geçerli , başarılı bir standart oluşturmuştur . <EOS>
Prediction: sinif , havuz ve açık deniz çalışmalarıyla , tüm dünyada geçerli , başarılı bir standart oluşturmuştur . 

Example 2
Input: bu standart , sualtinda kendini rahat hisseden herkesin , sportif dalici olabilecegini , bunun icin fiziki guc ve cok iyi yuzme bilmenin sart olmadigini savunur . <EOS>
Label: bu standart , sualtında kendini rahat hisseden herkesin , sportif dalıcı olabileceğini , bunun için fiziki güç ve cok iyi yüzme bilmenin şart olmadığını savunur . <EOS>
Prediction: bu standart , sualtında kendini rahat hisseden herkesin , sportif dalıcı olabileceğini , bunun için fiziki güç ve çok iyi yüzme bilmenin sart olmadığını savunur . 

Example 3
Input: yapilan arastirmalar , ogre

In [17]:
# Load the test dataset
test_data = pd.read_csv('/content/drive/My Drive/test.csv')

test_data['Sentence'] = test_data['Sentence'].apply(remove_first_char)

test_data['Sentence'] = test_data['Sentence'].str.lower()

# Tokenize test sentences
test_data['tokenized_input'] = test_data['Sentence'].apply(letter_tokenization)

test_data['tokenized_input'] = test_data['tokenized_input'].apply(lambda x: x + ['<EOS>'])

# Convert test sentences to indices
test_data['indexed_input'] = test_data['tokenized_input'].apply(tokens_to_indices)

# Pad test sequences
padded_test_input = pad_sequences(test_data['indexed_input'], maxlen=max_length, padding='post')

# Predict labels for test data
test_predictions = model.predict(padded_test_input)

# Convert predicted indices to sentences
predicted_sentences = []
for prediction in test_predictions:
    predicted_sentence = []
    for i in prediction.argmax(axis=1):
        if i == 0:  # Stop at padding
            break
        if index_to_char[i] == '<EOS>':  # Stop at <EOS>
            break
        predicted_sentence.append(index_to_char[i])
    predicted_sentences.append(''.join(predicted_sentence))

# Add predicted sentences to test_data
test_data['Predicted_Sentence'] = predicted_sentences

37/37 [==============================] - 0s 9ms/step


In [18]:
test_data.head(30)

,ID,Sentence,tokenized_input,indexed_input,Predicted_Sentence
0,0,tr ekonomi ve politika haberleri turkiye nin e...,"[t, r, , e, k, o, n, o, m, i, , v, e, , p, ...","[22, 20, 7, 14, 16, 25, 5, 25, 19, 4, 7, 11, 1...",tr ekonomi ve politika haberleri türkiye nin e...
1,1,uye girisi,"[u, y, e, , g, i, r, i, s, i, , <EOS>]","[12, 21, 14, 7, 23, 4, 20, 4, 3, 4, 7, 2]",uye girişi
2,2,son guncelleme 12:12,"[s, o, n, , g, u, n, c, e, l, l, e, m, e, , ...","[3, 25, 5, 7, 23, 12, 5, 15, 14, 18, 18, 14, 1...",son güncelleme 12:12
3,3,imrali mit gorusmesi ihtiyac duyuldukca oluyor,"[i, m, r, a, l, i, , m, i, t, , g, o, r, u, ...","[4, 19, 20, 10, 18, 4, 7, 19, 4, 22, 7, 23, 25...",imrali mit görüşmesi ihtiyac düyuldukça oluyor
4,4,suriye deki silahli selefi muhalifler yeni kur...,"[s, u, r, i, y, e, , d, e, k, i, , s, i, l, ...","[3, 12, 20, 4, 21, 14, 7, 17, 14, 16, 4, 7, 3,...",suriye deki silahlı selefi mühalifler yeni kur...
5,5,ancak olum haberleri savastan cok tek tarafli ...,"[a, n, c, a, k, , o, l, u, m, , h, a, b, e, ...","[10, 5, 15, 10, 16, 7, 25, 18, 12, 19, 7, 9, 1...",ancak olum haberleri savaştan çok tek taraflı ...
6,6,israil in 4 uncu gunune giren gazze saldirilar...,"[i, s, r, a, i, l, , i, n, , 4, , u, n, c, ...","[4, 3, 20, 10, 4, 18, 7, 4, 5, 7, 31, 7, 12, 5...",işrail in 4 uncü gününe giren gazze saldırılar...
7,7,serbes: memecan sen mizahci misin,"[s, e, r, b, e, s, :, , m, e, m, e, c, a, n, ...","[3, 14, 20, 24, 14, 3, 49, 7, 19, 14, 19, 14, ...",serbes: memecan sen mizahçı misin
8,8,muslum gurses yogun bakimda,"[m, u, s, l, u, m, , g, u, r, s, e, s, , y, ...","[19, 12, 3, 18, 12, 19, 7, 23, 12, 20, 3, 14, ...",müslum gürses yoğun bakımda
9,9,takip et: wwwradikalcomtr,"[t, a, k, i, p, , e, t, :, , w, w, w, r, a, ...","[22, 10, 16, 4, 27, 7, 14, 22, 49, 7, 54, 54, ...",takıp et: wwwradikalçomtr
